# Simulation in air

# Purpose
Check the equation of motion

# Methodology
Conduct some simulations to see that the object is behaving in air.

# Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.plot import track_plot
from vessel_manoeuvring_models.equation import Equation

## Load models:
# (Uncomment these for faster loading):
#import vessel_manoeuvring_models.models.vmm_abkowitz  as vmm_abkowitz 
import vessel_manoeuvring_models.models.vmm_air as vmm_air

In [ ]:
Math(vlatex(vmm_air.X_eq))

In [ ]:
Math(vlatex(vmm_air.Y_eq))

In [ ]:
Math(vlatex(vmm_air.N_eq))

In [ ]:
ship_parameters = pd.Series({
        
        'L' : 5,
        'B' : 1,
        'x_G' : 0,  # motions are expressed at CG
        'm' : 2,
        'I_z': 2, 
    })

parameters = {}


In [ ]:
t_ = np.linspace(0,10,1000)
df = pd.DataFrame(index=t_)
f_ext_x_ = 2
a_x_ = f_ext_x_/ship_parameters.m

df['u'] = a_x_*t_
df['v'] = 0
df['r'] = 0
df['x0'] = a_x_*t_**2/2
df['y0'] = 0
df['psi'] = 0
df['beta'] = -np.arctan2(df['v'],df['u'])
df['f_ext_x']=f_ext_x_
df['f_ext_y']=0
df['m_ext_z']=0

result_air = vmm_air.simulator.simulate(df_=df, 
                                        parameters=parameters, 
                                        ship_parameters=ship_parameters, 
                                        control_keys=['f_ext_x','f_ext_y','m_ext_z'])
result_air.plot_compare()

In [ ]:
t_ = np.linspace(0,10,1000)
df = pd.DataFrame(index=t_)
V_=6
r_ = 2*np.pi/t_[-1]

f_ext = ship_parameters.m*r_*V_



df['beta'] = np.deg2rad(30)
df['u'] = V_*np.cos(df['beta'])
df['v'] = -V_*np.sin(df['beta'])
df['r'] = r_

R_=V_*t_[-1]/(2*np.pi)

phi_ = r_*t_
df['psi'] = phi_ + df['beta']
df['x0'] = R_*np.sin(phi_)
df['y0'] = R_-R_*np.cos(phi_)


df['f_ext_x']=f_ext*np.sin(df['beta'])
df['f_ext_y']=f_ext*np.cos(df['beta'])
df['m_ext_z']=0

result_air = vmm_air.simulator.simulate(df_=df, 
                                        parameters=parameters, 
                                        ship_parameters=ship_parameters, 
                                        control_keys=['f_ext_x','f_ext_y','m_ext_z'])
result_air.plot_compare()